In [1]:
pip install fairlearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Libraries
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.feature_selection import SelectFromModel


from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout,BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report, precision_score, recall_score, roc_auc_score, confusion_matrix, accuracy_score

import matplotlib.pyplot as plt
from scipy.stats import ranksums
import statistics
import math
from scipy.interpolate import interp1d
from numpy.random import seed
seed(42)
from tensorflow import random
random.set_seed(42)


In [4]:
#Variables
threshold = 0.15
num_samples = 100
specified_specificity = 0.8

In [5]:
#Basic Functions
def preprocess_data(df_train, df_test): #could be df_train and df_val during CV
    #Standardize the data and impute nan data- based on training, create pipeline
    if not df_train.select_dtypes(include=['category', 'int']).empty:
        print('Warning: There is categorical data in your training data-set')
    num_cols = df_train.select_dtypes(include=['float']).columns
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('normalizer', Normalizer())
        ])
    df_train = pipeline.fit_transform(df_train)
    df_test = pipeline.transform(df_test)
    return(df_train, df_test)



def train(df_train, dfy_train):
    clf = RandomForestClassifier(criterion='gini', bootstrap=(True), class_weight='balanced_subsample', random_state=42)
    clf_trained = clf.fit(df_train, dfy_train)
    return(clf_trained)

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from plotly.subplots import make_subplots

def test(clf_trained, df_test, dfy_test, threshold, test=False):
    proba_test = clf_trained.predict(df_test, verbose=0)
    y_pred_test = proba_test[:,1] > threshold

    tn, fp, fn, tp = confusion_matrix(dfy_test[:,1], y_pred_test).ravel()
    sen = tp / (tp + fn)
    spe = tn / (tn + fp)
    FNR = fn / (tp + fn)
    if test==True:
        return(sen, spe, FNR, y_pred_test, proba_test[:,1])
    return(sen, spe, FNR)


def evaluate_disparities(clf, proba_test, X_test, y_test, test_ind, ind_test_b, ind_test_nb):
  np.random.seed(0)
  y_test_df = pd.DataFrame(data=y_test[:,1], index=test_ind)
  #Get bootstrapped sen, spe, FNR test array for df_b:
  proba_test_b = pd.DataFrame(data=proba_test, index=test_ind).loc[ind_test_b,:].values
  df_test_b = pd.DataFrame(data=X_test, columns=cols, index=test_ind).loc[ind_test_b,:].values
  dfy_test_b = y_test_df.loc[ind_test_b].values
  df_sen_b_b = []
  df_spe_b_b = []
  df_tpr_b_b = []
  df_auc_b_b = []
  for i in np.arange(num_samples):
    idx = np.random.randint(0, df_test_b.shape[0], (df_test_b.shape[0],))
    #sen_bootstrap_b, spe_bootstrap_b, FNR_bootstrap_b = test(clf, df_test_b[idx,:], dfy_test_b[idx], threshold)
    fpr_bootstrap_b, tpr_bootstrap_b, _ = roc_curve(dfy_test_b[idx], proba_test_b[idx])
    tpr_at_target_fpr = np.interp(0.2, fpr_bootstrap_b, tpr_bootstrap_b)
    auc_bootstrap_b = roc_auc_score(dfy_test_b[idx], proba_test_b[idx])
    #df_sen_b_b.append(sen_bootstrap_b)
    #df_spe_b_b.append(spe_bootstrap_b)
    df_tpr_b_b.append(tpr_at_target_fpr)
    df_auc_b_b.append(auc_bootstrap_b)
  sorted = np.sort(df_tpr_b_b)
  tpr_lower_bound_b = np.percentile(sorted, (1 - 0.95) / 2 * 100)
  tpr_upper_bound_b = np.percentile(sorted, (1 + 0.95) / 2 * 100)
  sorted = np.sort(df_auc_b_b)
  auc_lower_bound_b = np.percentile(sorted, (1 - 0.95) / 2 * 100)
  auc_upper_bound_b = np.percentile(sorted, (1 + 0.95) / 2 * 100)

  #Get bootstrapped sen, spe, FNR test array for df_nb:
  proba_test_nb = pd.DataFrame(data=proba_test, index=test_ind).loc[ind_test_nb,:].values
  df_test_nb = pd.DataFrame(data=X_test, columns=cols, index=test_ind).loc[ind_test_nb,:].values
  dfy_test_nb = y_test_df.loc[ind_test_nb].values
  df_sen_b_nb = []
  df_spe_b_nb = []
  df_tpr_b_nb = []
  df_auc_b_nb = []
  for i in np.arange(num_samples):
    idx = np.random.randint(0, df_test_nb.shape[0], (df_test_nb.shape[0],))
    #sen_bootstrap_nb, spe_bootstrap_nb, FNR_bootstrap_nb = test(clf, df_test_nb[idx,:], dfy_test_nb[idx], threshold)
    fpr_bootstrap_nb, tpr_bootstrap_nb, _ = roc_curve(dfy_test_nb[idx], proba_test_nb[idx])
    tpr_at_target_fpr = np.interp(0.2, fpr_bootstrap_nb, tpr_bootstrap_nb)
    auc_bootstrap_nb = roc_auc_score(dfy_test_nb[idx], proba_test_nb[idx])
    #df_sen_b_nb.append(sen_bootstrap_nb)
    #df_spe_b_nb.append(spe_bootstrap_nb)
    df_tpr_b_nb.append(tpr_at_target_fpr)
    df_auc_b_nb.append(auc_bootstrap_nb)

  sorted = np.sort(df_tpr_b_nb)
  tpr_lower_bound_nb = np.percentile(sorted, (1 - 0.95) / 2 * 100)
  tpr_upper_bound_nb = np.percentile(sorted, (1 + 0.95) / 2 * 100)
  sorted = np.sort(df_auc_b_nb)
  auc_lower_bound_nb = np.percentile(sorted, (1 - 0.95) / 2 * 100)
  auc_upper_bound_nb = np.percentile(sorted, (1 + 0.95) / 2 * 100)

  #Comparison df no sex female vs df no sex male: test
  #_, pvalue11 = ranksums(x=df_sen_b_b, y=df_sen_b_nb)
  #_, pvalue12 = ranksums(x=df_spe_b_b, y=df_spe_b_nb)
  _, pvalue13 = ranksums(x=df_auc_b_b, y=df_auc_b_nb)
  print('Comparison df black vs df non-black for AUC:', format(pvalue13,"e"))

  #Final ROC plotting
  proba_test_df = pd.DataFrame(data=proba_test, index=test_ind)
  y_test = pd.DataFrame(data=y_test[:,1], index = test_ind)

  #ROC Black
  fpr_b, tpr_b, _ = roc_curve(y_test.loc[ind_test_b], proba_test_df.loc[ind_test_b])
  roc_auc_score_b = roc_auc_score(y_test.loc[ind_test_b], proba_test_df.loc[ind_test_b])
  interp_func = interp1d(fpr_b, tpr_b)
  specified_sensitivity = interp_func(1 - specified_specificity)



  #ROC Non-Black
  fpr_nb, tpr_nb, _ = roc_curve(y_test.loc[ind_test_nb], proba_test_df.loc[ind_test_nb])
  roc_auc_score_nb = roc_auc_score(y_test.loc[ind_test_nb], proba_test_df.loc[ind_test_nb])
  interp_func = interp1d(fpr_nb, tpr_nb)
  specified_sensitivity = interp_func(1 - specified_specificity)
  return pvalue13, auc_lower_bound_b, auc_upper_bound_b, auc_lower_bound_nb, auc_upper_bound_nb, tpr_lower_bound_b, tpr_upper_bound_b, tpr_lower_bound_nb, tpr_upper_bound_nb


def difference(proba_test, X_test, y_test, test_ind, ind_test_b, ind_test_nb):
  y_test = pd.DataFrame(data=y_test[:,1], index = test_ind)
  proba_test_df = pd.DataFrame(data=proba_test, index=test_ind)
  #ROC Black
  fpr_b, tpr_b, _ = roc_curve(y_test.loc[ind_test_b], proba_test_df.loc[ind_test_b])
  roc_auc_score_b = roc_auc_score(y_test.loc[ind_test_b], proba_test_df.loc[ind_test_b])
  interp_func = interp1d(fpr_b, tpr_b)
  specified_sensitivity_b = interp_func(1 - specified_specificity)


  #ROC Non-Black
  fpr_nb, tpr_nb, _ = roc_curve(y_test.loc[ind_test_nb], proba_test_df.loc[ind_test_nb])
  roc_auc_score_nb = roc_auc_score(y_test.loc[ind_test_nb], proba_test_df.loc[ind_test_nb])
  interp_func = interp1d(fpr_nb, tpr_nb)
  specified_sensitivity_nb = interp_func(1 - specified_specificity)

  return specified_sensitivity_nb - specified_sensitivity_b

class CustomLoss(tf.keras.losses.Loss):
  def __init__(self, predictor, adversary, y_race, X_ad, a, b):
      super().__init__()
      self.predictor = predictor
      self.adversary = adversary
      self.y_race = y_race
      self.X_ad = X_ad
      self.a = a
      self.b = b

  def call(self, y_true, y_pred):
      predictor_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)

      batch_size = tf.shape(y_pred)[0]
      idx = tf.random.uniform([batch_size], minval=0, maxval=tf.shape(self.X_ad)[0], dtype=tf.int32)
      X_ad = tf.gather(self.X_ad, idx)

      y_race = tf.gather(self.y_race, idx)

      adversary_pred = self.adversary(X_ad)
      adversary_loss = tf.keras.losses.binary_crossentropy(y_race, adversary_pred)


      combined_loss = predictor_loss - self.b*predictor_loss/adversary_loss - self.a*adversary_loss

      return combined_loss


In [ ]:
from joblib import load

X_test = "External validation features"
y_test = "External validation labels (ACS = 1, no ACS = 0)"
y_test = y_test.astype(int)
ID_test = X_test.index

#Race indices
race_test = X_test['race']
race_test = race_test.astype(int)
ind_test_nb = X_test[X_test['race']==0].index
ind_test_b = X_test[X_test['race']==1].index


#Drop sensitive features
X_test.drop(['race'],axis = 1, inplace=True)

df = pd.read_excel('feature_importance_final.xlsx')
selected_features = df['feature'].tolist()
X_test = X_test[selected_features[:148]] #select top 148 features
cols = X_test.columns


scaler1 = load('scaler1.joblib')
X_test = scaler1.transform(X_test)
X_test = pd.DataFrame(data=X_test, columns = cols, index=ID_test)


#Top features selected by RF
top_indices = np.load('rf_chosen_100features.npy', allow_pickle=True).tolist()
X_test = X_test[top_indices]
cols = X_test.columns

scaler2 = load('scaler2.joblib')
X_test = scaler2.transform(X_test)
y_test = to_categorical(y_test, num_classes=2)


model_p = tf.keras.models.load_model('model_p.h5', compile=False)


print('Testing')
sen, spe, FNR, y_pred_test, proba_test  = test(model_p, X_test, y_test, threshold, test=True)
print('Test sen = ', round(sen,4),'\n')
print('Test spe = ', round(spe,4))
pvalue, auc_lower_b, auc_upper_b, auc_lower_nb, auc_upper_nb, tpr_lower_b, tpr_upper_b, tpr_lower_nb, tpr_upper_nb = evaluate_disparities(model_p, proba_test, X_test, y_test, ID_test, ind_test_b, ind_test_nb)

#Final ROC plotting
proba_test_df = pd.DataFrame(data=proba_test, index=ID_test)
y_test = pd.DataFrame(data=y_test[:,1], index = ID_test)

#ROC Black
fpr_b, tpr_b, _ = roc_curve(y_test.loc[ind_test_b], proba_test_df.loc[ind_test_b])
roc_auc_score_b = roc_auc_score(y_test.loc[ind_test_b], proba_test_df.loc[ind_test_b])
interp_func = interp1d(fpr_b, tpr_b)
specified_sensitivity = interp_func(1 - specified_specificity)
print("Black Sensitivity at 80% Specificity:", specified_sensitivity)


#ROC Non-Black
fpr_nb, tpr_nb, _ = roc_curve(y_test.loc[ind_test_nb], proba_test_df.loc[ind_test_nb])
roc_auc_score_nb = roc_auc_score(y_test.loc[ind_test_nb], proba_test_df.loc[ind_test_nb])
interp_func = interp1d(fpr_nb, tpr_nb)
specified_sensitivity = interp_func(1 - specified_specificity)
print("Non-Black Sensitivity at 80% Specificity:", specified_sensitivity)

pvalue = round(pvalue, 2)
plt.figure(figsize=[5, 5])
plt.plot(fpr_nb, tpr_nb, color='b',
              label=r'Non-Black AUC = %0.3f (%0.3f - %0.3f)' % (roc_auc_score_nb, auc_lower_nb, auc_upper_nb),
              lw=2, alpha=.8)
index_nb = (np.abs(fpr_nb - 0.2)).argmin()
plt.scatter(fpr_nb[index_nb], tpr_nb[index_nb], lw = .2, color='black')
plt.plot(fpr_b, tpr_b, color='#d62728',
              label=r'Black AUC = %0.3f (%0.3f - %0.3f)' % (roc_auc_score_b, auc_lower_b, auc_upper_b),
              lw=2, alpha=.8)
index_b = (np.abs(fpr_b - 0.2)).argmin()
plt.scatter(fpr_b[index_b], tpr_b[index_b], lw = .2, color='black', label=f'FPR = 0.20')
if pvalue > 0.001:
  plt.text(0.05, 0.95, f'p-value: {pvalue}', fontsize=11, transform=plt.gca().transAxes, color='black')
else:
  plt.text(0.05, 0.95, f'p << 0.001', fontsize=11, transform=plt.gca().transAxes, color='black')
plt.plot([0, 1], [0, 1], linestyle='--', lw=1, color='grey', alpha=.5)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate = 1 - Specificity')
plt.ylabel('True Positive Rate = Sensitivity')
plt.legend(loc="lower right")
plt.savefig('Figure_AD.png', dpi=1200, bbox_inches='tight')
plt.show()

print(f"Black TPR: {tpr_lower_b}-{tpr_upper_b}")
print(f"Non-Black TPR: {tpr_lower_nb}-{tpr_upper_nb}")

In [ ]:
from fairlearn.metrics import equalized_odds_ratio
from fairlearn.metrics import demographic_parity_ratio
from fairlearn.metrics import equal_opportunity_ratio

fpr, tpr, thresholds = roc_curve(y_test, proba_test)
target_fpr = 0.20
idx = np.argmin(np.abs(fpr - 0.2))
print("Index:", idx, "FPR:", fpr[idx], "Threshold:", thresholds[idx])
thre = thresholds[idx]


y_pred_test = (proba_test >= thre).astype(int)
race_test = to_categorical(race_test, num_classes=2)

dp = demographic_parity_ratio(
    y_test,
    y_pred_test,
    sensitive_features=race_test[:,1]
)
print("Demographic Parity Difference:", dp)
eod = equalized_odds_ratio(
    y_test,
    y_pred_test,
    sensitive_features=race_test[:,1]
)

print("Equalized Odds Difference:", eod)

eop = equal_opportunity_ratio(
    y_test,
    y_pred_test,
    sensitive_features=race_test[:,1]
)
print("Equal Opportunity Difference:", eop)